<a href="https://colab.research.google.com/github/minerado/ml/blob/main/gpt-implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT Implementation

This implementation is based on Andej Karpathy's tutorial [Let's build GPT](https://www.youtube.com/watch?v=kCc8FmEb1nY)

For this exercise, we'll be using the tinyshakespare data as vocabulary:

In [5]:
# Fetch and save data
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [7]:
with open('input.txt', 'r') as f:
    text = f.read()

In [10]:
print(f"Length of dataset: {len(text)} characters")

Length of dataset: 1115394 characters


In [11]:
# Let's inspect it
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [19]:
# Unique characters to appear in the data
chars = sorted(set(text))
vocab_size = len(chars)

print(f"Vocabulary: {''.join(chars)}")
print(f"Vocabulary size: {vocab_size}")

Vocabulary: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


In [32]:
# Create mapping from characters to integers
char_dict = {char: i for i, char in enumerate(chars)}
index_dict = {i: char for i, char in enumerate(chars)}

def encode(s): return [char_dict[c] for c in s]
def decode(lst): return ''.join([index_dict[i] for i in lst])

In [35]:
decode(encode("hello world"))

'hello world'